
# 01 — EEG Visualization & Simple Spectral Analysis

**Goal:** Provide a minimal, well-documented example to visualize an EEG-like signal and compute a simple spectral analysis (FFT & bandpower).  
**Why:** Recruiters and collaborators should see a working, readable notebook on day one.

> This notebook uses a *synthetic* EEG-like signal (no protected health data). You can later replace the synthetic generator with a real dataset loader.


In [ ]:

# Imports (no internet needed)
import numpy as np
import matplotlib.pyplot as plt

# Utility: ensure plots are inline in Jupyter
# (Most Jupyter environments default to inline. Left here for clarity.)
# %matplotlib inline



## 1. Generate an EEG-like signal (synthetic)

We simulate 10 seconds at 256 Hz, mixing common EEG rhythms:
- Delta (2 Hz), Theta (6 Hz), Alpha (10 Hz), Beta (20 Hz), plus a weak 60 Hz interference.
- Add Gaussian noise to resemble real-world recordings.


In [ ]:

fs = 256            # Sampling frequency (Hz)
duration = 10       # seconds
t = np.arange(0, duration, 1/fs)

# Component amplitudes (arbitrary units)
amp_delta = 20.0
amp_theta = 10.0
amp_alpha = 15.0
amp_beta  = 8.0
amp_line  = 2.0     # 60 Hz line noise

# Build components
delta = amp_delta * np.sin(2*np.pi*2*t)
theta = amp_theta * np.sin(2*np.pi*6*t + 0.3)
alpha = amp_alpha * np.sin(2*np.pi*10*t + 1.0)
beta  = amp_beta  * np.sin(2*np.pi*20*t + 2.0)
line  = amp_line  * np.sin(2*np.pi*60*t)

# Additive Gaussian noise
rng = np.random.default_rng(42)
noise = rng.normal(0, 3.0, t.shape)

eeg = delta + theta + alpha + beta + line + noise

eeg[:10], t[:10]



## 2. Plot the time-domain signal

We display the first 5 seconds to make patterns easier to see.


In [ ]:

seconds_to_show = 5
N = int(seconds_to_show * fs)

plt.figure(figsize=(10, 4))
plt.plot(t[:N], eeg[:N])
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (a.u.)")
plt.title("Synthetic EEG-like Signal (first 5 s)")
plt.tight_layout()
plt.show()



## 3. Compute the amplitude spectrum (FFT)

A basic Fast Fourier Transform (FFT) to reveal dominant frequencies.


In [ ]:

# Compute one-sided FFT
n = len(eeg)
freqs = np.fft.rfftfreq(n, d=1/fs)
fft_vals = np.fft.rfft(eeg)
amp_spectrum = np.abs(fft_vals) * 2 / n  # amplitude scaling

plt.figure(figsize=(10, 4))
plt.plot(freqs, amp_spectrum)
plt.xlim(0, 80)  # focus on 0–80 Hz
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.title("Amplitude Spectrum (FFT)")
plt.tight_layout()
plt.show()



## 4. Simple bandpower

A compact function to estimate bandpower by integrating the amplitude spectrum within a frequency band.


In [ ]:

def bandpower(freqs, amp, fmin, fmax):
    """Approximate bandpower by integrating the amplitude in [fmin, fmax]."""
    mask = (freqs >= fmin) & (freqs <= fmax)
    # Trapezoidal integration
    return np.trapz(amp[mask], freqs[mask])

bands = {
    "Delta (0.5–4 Hz)": (0.5, 4),
    "Theta (4–8 Hz)": (4, 8),
    "Alpha (8–13 Hz)": (8, 13),
    "Beta (13–30 Hz)": (13, 30),
    "Line (57–63 Hz)": (57, 63),
}

results = {name: bandpower(freqs, amp_spectrum, lo, hi) for name, (lo, hi) in bands.items()}
results



## 5. Visualize bandpower

A simple bar plot to summarize energy per band.


In [ ]:

names = list(results.keys())
values = [results[k] for k in names]

plt.figure(figsize=(8, 4))
plt.bar(names, values)
plt.ylabel("Approx. Bandpower (a.u.)")
plt.title("Bandpower by EEG Band")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()



## 6. Replace with real data (when ready)

1. Export a single EEG channel (e.g., `.csv` with columns `time, voltage`).
2. Replace the synthetic generator with a file loader:
   ```python
   import pandas as pd
   df = pd.read_csv("path/to/eeg_channel.csv")
   t = df["time"].to_numpy()
   eeg = df["voltage"].to_numpy()
   fs = 1.0 / np.mean(np.diff(t))
   ```
3. Re-run sections **2–5** unchanged.



## 7. Mini roadmap

- Add Welch PSD for smoother spectra.
- Load open EEG datasets (e.g., TUH, CHB-MIT) and plot multi-channel segments.
- Train a simple classifier (e.g., logistic regression or 1D CNN) for normal vs. epileptiform windows.
- Wrap utilities into a tiny `eeg_utils.py` module.
